# Part 4.6. PackedSequence

## 1. Examples of Sequential Data
`text`나 `voice` 데이터가 있다. 이러한 Sequential Data는 길이가 미정인 데이터가 많다.

![22-1.png](./img/22-1.png)

## 2. How to Make a Batch
### 2.1. Padding Method
가장 긴 sequence의 길이에 맞춰 나머지 데이터들의 뒷 부분들을 `pad`라는 토큰으로 채워넣는 방법이다. 다만 계산하지 않아도 될 부분을 채워야 한다는 단점이 있다.

![22-2.png](./img/22-2.png)

### 2.2. Packing Method
sequence의 길이에 대한 정보를 저장하는 방식. 다만 길이 내림차순으로 문장을 정렬해야 하고 **Padding Method**에 비해 구현이 더 복잡하다는 단점이 있다.

![22-3.png](./img/22-3.png)

![22-4.png](./img/22-4.png)

### 2.3. Pytorch Functions
**padding**과 **packing**에 대한 4가지 파이토치 라이브러리 함수들에 대한 관계도이다.
* `pack_sequence` : tensor를 packSequence로
* `pad_sequence` : tensor를 paddedSequence로
* `pad_packed_sequence` : packedSequence를 paddedSequence로
* `pack_padded_sequence` : paddedSequence를 packedSeqence로

![22-5.png](./img/22-5.png)

## 3. Implementation


### 3.1. 필요한 모듈 임포트

In [0]:
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

### 3.2. 예제 데이터셋

In [2]:
# 임의의 문장으로 구성된 데이터셋
data = ['hello world',
        'midnight',
        'calculation',
        'path',
        'short circuit']

# 딕셔너리 생성
char_set = ['<pad>'] + list(set(char for seq in data for char in seq))  # <pad> 토큰을 포함한 문자들 생성
char2idx = {char: idx for idx, char in enumerate(char_set)}             # 인덱싱
print('char_set:', char_set)
print('char_set length:', len(char_set))

char_set: ['<pad>', 'h', 'r', 'l', 't', 'n', 'g', 'e', 'c', 'w', 'u', 'a', 'm', ' ', 'p', 's', 'i', 'd', 'o']
char_set length: 19


In [3]:
# 문자를 인덱스로 변환한 뒤 tensor로 변환
X = [torch.LongTensor([char2idx[char] for char in seq]) for seq in data]

for sequence in X:
    print(sequence)

tensor([ 1,  7,  3,  3, 18, 13,  9, 18,  2,  3, 17])
tensor([12, 16, 17,  5, 16,  6,  1,  4])
tensor([ 8, 11,  3,  8, 10,  3, 11,  4, 16, 18,  5])
tensor([14, 11,  4,  1])
tensor([15,  1, 18,  2,  4, 13,  8, 16,  2,  8, 10, 16,  4])


In [4]:
# sequence의 길이
lengths = [len(seq) for seq in X]
print('lengths:', lengths)

lengths: [11, 8, 11, 4, 13]


### 3.3. paddedSequence와 packedSequence 생성

In [5]:
# paddedSequence 생성
padded_sequence = pad_sequence(X, batch_first=True)
print(padded_sequence)
print(padded_sequence.shape)

tensor([[ 1,  7,  3,  3, 18, 13,  9, 18,  2,  3, 17,  0,  0],
        [12, 16, 17,  5, 16,  6,  1,  4,  0,  0,  0,  0,  0],
        [ 8, 11,  3,  8, 10,  3, 11,  4, 16, 18,  5,  0,  0],
        [14, 11,  4,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [15,  1, 18,  2,  4, 13,  8, 16,  2,  8, 10, 16,  4]])
torch.Size([5, 13])


In [7]:
# 내림차순으로 문장 정렬
sorted_idx = sorted(range(len(lengths)), key=lengths.__getitem__, reverse=True)
sorted_X = [X[idx] for idx in sorted_idx]

for sequence in sorted_X:
    print(sequence)

tensor([15,  1, 18,  2,  4, 13,  8, 16,  2,  8, 10, 16,  4])
tensor([ 1,  7,  3,  3, 18, 13,  9, 18,  2,  3, 17])
tensor([ 8, 11,  3,  8, 10,  3, 11,  4, 16, 18,  5])
tensor([12, 16, 17,  5, 16,  6,  1,  4])
tensor([14, 11,  4,  1])


In [8]:
# packed_sequence 생성
packed_sequence = pack_sequence(sorted_X)
print(packed_sequence)

PackedSequence(data=tensor([15,  1,  8, 12, 14,  1,  7, 11, 16, 11, 18,  3,  3, 17,  4,  2,  3,  8,
         5,  1,  4, 18, 10, 16, 13, 13,  3,  6,  8,  9, 11,  1, 16, 18,  4,  4,
         2,  2, 16,  8,  3, 18, 10, 17,  5, 16,  4]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]), sorted_indices=None, unsorted_indices=None)


### 3.4. Embedding 적용

In [9]:
# char 개수만큼의 정방행렬 생성
eye = torch.eye(len(char_set))
embedded_tensor = eye[padded_sequence]

# (batch_size, max_sequence_length, number_of_input_tokens)
print(embedded_tensor.shape) 

torch.Size([5, 13, 19])


In [10]:
# packedSequence를 embedding
embedded_packed_seq = pack_sequence([eye[X[idx]] for idx in sorted_idx])
print(embedded_packed_seq.data.shape)

torch.Size([47, 19])


### 3.5. RNN 모델 정의

In [0]:
# 모델 정의
rnn = torch.nn.RNN(input_size=len(char_set), hidden_size=30, batch_first=True)

In [13]:
rnn_output, hidden = rnn(embedded_tensor)

print(rnn_output.shape)     # (batch_size, max_seq_length, hidden_size)
print(hidden.shape)         # (num_layers * num_directions, batch_size, hidden_size)

torch.Size([5, 13, 30])
torch.Size([1, 5, 30])


In [14]:
rnn_output, hidden = rnn(embedded_packed_seq)

print(rnn_output.data.shape)
print(hidden.data.shape)

torch.Size([47, 30])
torch.Size([1, 5, 30])
